In [ ]:
import os
import ast
import torch
import pandas as pd
from torch import nn
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader, random_split
from efficientnet_pytorch import EfficientNet
import flwr as fl

# ---------------------- Paths ----------------------
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")
csv_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/EfficientNet/ocular_labels.csv"

# ---------------------- Load Data ----------------------
if not os.path.exists(csv_path):
    df = pd.read_excel(data_file)
    label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]
    rows = []
    for _, r in df.iterrows():
        rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
        rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    pd.DataFrame(rows).to_csv(csv_path, index=False)

df_images = pd.read_csv(csv_path)
df_images["labels"] = df_images["labels"].apply(ast.literal_eval)

# ---------------------- Dataset Split ----------------------
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# ---------------------- Dataset Class ----------------------
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform or T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# ---------------------- Model ----------------------
def get_model():
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(model._fc.in_features, 8)
    return model

# ---------------------- Device ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------------- Flower Client ----------------------
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader, cid):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.client_id = cid
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.0]*8).to(device))
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.round_counter = 0

    def get_parameters(self, config): return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        self.model.load_state_dict(new_state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for _ in range(3):
            for x, y in self.train_loader:
                x, y = x.to(device), y.to(device)
                self.optimizer.zero_grad()
                loss = self.criterion(self.model(x), y)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters({}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        y_true, y_pred, val_loss = [], [], 0.0
        with torch.no_grad():
            for x, y in self.val_loader:
                x, y = x.to(device), y.to(device)
                out = self.model(x)
                val_loss += self.criterion(out, y).item()
                preds = torch.sigmoid(out).cpu().numpy()
                y_true.extend(y.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)
        self.round_counter += 1

        path = f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/EfficientNet/FL_Client_{self.client_id}.csv"
        pd.DataFrame([{
            "client": self.client_id, "round": self.round_counter,
            "loss": val_loss / len(self.val_loader), "accuracy": acc,
            "precision": prec, "recall": rec, "f1": f1
        }]).to_csv(path, mode="a", header=not os.path.exists(path), index=False)

        print(f"Client {self.client_id} - R{self.round_counter} | Acc: {acc:.4f} | F1: {f1:.4f}")
        return float(val_loss / len(self.val_loader)), len(self.val_loader.dataset), {}

# ---------------------- Client Function ----------------------
def client_fn(cid: str):
    idx = int(cid)
    df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(df, train_images_dir)
    train_len = int(0.8 * len(dataset))
    train_ds, val_ds = random_split(dataset, [train_len, len(dataset) - train_len])
    return FlowerClient(get_model(), DataLoader(train_ds, batch_size=8, shuffle=True),
                        DataLoader(val_ds, batch_size=8), cid).to_client()

# ---------------------- Global Evaluation ----------------------
def global_evaluate(server_round, parameters):
    model = get_model().to(device)
    state_dict = model.state_dict()
    new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
    model.load_state_dict(new_state_dict)
    model.eval()

    all_y_true, all_y_pred, total_loss = [], [], 0.0
    with torch.no_grad():
        for idx in range(5):
            df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
            dataset = OcularDataset(df, train_images_dir)
            _, val_ds = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
            val_loader = DataLoader(val_ds, batch_size=8)
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                total_loss += nn.BCEWithLogitsLoss()(out, y).item()
                preds = torch.sigmoid(out).cpu().numpy()
                all_y_true.extend(y.cpu().numpy())
                all_y_pred.extend((preds > 0.3).astype(int))

    acc = accuracy_score(all_y_true, all_y_pred)
    prec = precision_score(all_y_true, all_y_pred, average="samples", zero_division=0)
    rec = recall_score(all_y_true, all_y_pred, average="samples", zero_division=0)
    f1 = f1_score(all_y_true, all_y_pred, average="samples", zero_division=0)
    loss = total_loss / len(all_y_true)

    print(f"🌍 Global - Round {server_round} - Acc: {acc:.4f} | F1: {f1:.4f}")

    pd.DataFrame([{
        "round": server_round, "loss": loss, "accuracy": acc,
        "precision": prec, "recall": rec, "f1": f1
    }]).to_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/EfficientNet/FL_Global.csv", mode="a", header=not os.path.exists("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/EfficientNet/FL_Global.csv"), index=False)

    return loss, {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# ---------------------- Strategy + Run ----------------------
strategy = fl.server.strategy.FedAvg(
    evaluate_fn=global_evaluate,
    fraction_fit=1.0,
    min_fit_clients=5,
    min_available_clients=5
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    client_resources={"num_cpus": 1, "num_gpus": 1.0},
    config=fl.server.ServerConfig(num_rounds=40),
)


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=40, no round_timeout
2025-07-04 11:42:44,093	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 24.0, 'object_store_memory': 16059860582.0, 'node:127.0.0.1': 1.0, 'memory': 32119721166.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1.0}
INFO :      Flo

(ClientAppActor pid=11708) Loaded pretrained weights for efficientnet-b0


(ClientAppActor pid=11708) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=11708) 
(ClientAppActor pid=11708)             This is a deprecated feature. It will be removed
(ClientAppActor pid=11708)             entirely in future versions of Flower.
(ClientAppActor pid=11708)         


(ClientAppActor pid=11708) Loaded pretrained weights for efficientnet-b0


(ClientAppActor pid=11708) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=11708) 
(ClientAppActor pid=11708)             This is a deprecated feature. It will be removed
(ClientAppActor pid=11708)             entirely in future versions of Flower.
(ClientAppActor pid=11708)         


...

(ClientAppActor pid=11708) Client 4 - R1 | Acc: 0.1250 | F1: 0.5930
(ClientAppActor pid=11708) Loaded pretrained weights for efficientnet-b0


(ClientAppActor pid=11708) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=11708) 
(ClientAppActor pid=11708)             This is a deprecated feature. It will be removed
(ClientAppActor pid=11708)             entirely in future versions of Flower.
(ClientAppActor pid=11708)         


(ClientAppActor pid=11708) Loaded pretrained weights for efficientnet-b0


(ClientAppActor pid=11708) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=11708) 
(ClientAppActor pid=11708)             This is a deprecated feature. It will be removed
(ClientAppActor pid=11708)             entirely in future versions of Flower.
(ClientAppActor pid=11708)         


(ClientAppActor pid=11708) Loaded pretrained weights for efficientnet-b0
